# Tennessee Example

This example is taken from [Wikipedia](https://en.wikipedia.org/wiki/Plurality_voting#Example)

Consider voting for the capital of Tennessee, assuming everyone in the state lives in one of four major cities.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

# Set up the votes as in the Wikipedia example.  Just assume 100 voters for simplicity.
n_voters = 100
choices = ['Memphis', 'Nashville','Chattanooga','Knoxville']
fraction = [.42, .26, .15, .17]
prefs = [[1, 2, 3, 4],
         [4, 1, 2, 3],
         [4, 3, 1, 2],
         [4, 3, 2, 1]]
votes = []
for f, p in zip(fraction, prefs):
    votes.extend(int(n_voters*f)*[p])
    
# Convert into a numpy array
votes = np.array(votes)

## Voting implementations

### First-past-the-post (FPTP) Voting
Simple plurality voting.  Plurality wins.  [Wikipedia](https://en.wikipedia.org/wiki/Plurality_voting).

In [2]:
from faculty_hiring import election

winner = election.choose_fptp(votes)

print('FPTP: Winner {}'.format(choices[winner[0]]))

FPTP: Winner Memphis


## Instant-Runoff Voting (IRV)

Remove candidate(s) with smallest number of 1st place votes and reassign their votes to the next ranked candidate until majority is reached.  [Wikipedia](https://en.wikipedia.org/wiki/Instant-runoff_voting).

In [3]:
winner = election.choose_irv(votes)
print('IRV: Winner {}'.format(choices[winner[0]]))

IRV: Winner Knoxville


## Condorcet Voting

The basic premise is to pick the candidate who is preferred in a head-to-head match up.  Ideally, the winner would always be preferred to every other candidate in a head to head election, but if that's not the case, at least pick the person for whom that's most often correct.  [Wikipedia](https://en.wikipedia.org/wiki/Condorcet_method).

Note: I'm not worrying about cycles or other strange situations with this method.  Maybe later?

In [4]:
winner = election.choose_condorcet(votes)
print('Condorcet: Winner {}'.format(choices[winner[0]]))

Condorcet: Winner Nashville


## Schulze Method of Voting

This is a Condorcet method that deals with the issue of cycles by removing the weakest defeats until a clear winner is found.  At least, that's what Wikipedia says.  It's hard to see how that works based on the implementation.  [Wikipedia](https://en.wikipedia.org/wiki/Schulze_method].

Since Schulze is a Condorcet method, should agree on the same winner as the Condorcet voting above.

In [5]:
winner = election.choose_schulze(votes)
print('Schulze: Winner {}'.format(choices[winner[0]]))

Schulze: Winner Nashville


Since Schulze is pretty complex to implement, let's check it on one more [Wikipedia example](https://en.wikipedia.org/wiki/Schulze_method#Example).

In [6]:
# Set up the votes as in the Wikipedia example.
choices2 = ['A', 'B','C','D','E']
votes2 = np.array(5*[[1,3,2,5,4]]+
                  5*[[1,5,4,2,3]]+
                  8*[[4,1,5,3,2]]+
                  3*[[2,3,1,5,4]]+
                  7*[[2,4,1,5,3]]+
                  2*[[3,2,1,4,5]]+
                  7*[[5,4,2,1,3]]+
                  8*[[3,2,5,4,1]])
winner = election.choose_schulze(votes2,5)
print('Second Schulze Example: Winner {}'.format(choices2[winner[0]]))
print('Ranking all the candidates: {}'.format('>'.join(choices2[i] for i in winner)))

Second Schulze Example: Winner E
Ranking all the candidates: E>A>C>B>D


## Borda Count Voting

This is a ranked voting approach based on points, so a form of positional voting.  [Wikipedia](https://en.wikipedia.org/wiki/Borda_count).

In [7]:
winner = election.choose_borda(votes)
print('Borda: Winner {}'.format(choices[winner[0]]))

Borda: Winner Nashville


## Score Voting

This is a cardinal voting system, meaning that voters score candidates on some numerical scale.  Candidates are scored independently of one another (i.e. they can have the same score if desired) and the scores are not a ranking.  For example, on a 5 points scale, a voter could give one candidate 5 points and the other two candidates 0 points if that reflected the voters opinions. [Wikipedia](https://en.wikipedia.org/wiki/Score_voting).

Note: for testing this code, we need to come up with a set of scores voters will assign to the options.  I'm using [these](https://en.wikipedia.org/wiki/Score_voting#Example).

In [8]:
ballots = [[10,  4,  2,  0],
           [ 0, 10,  4,  2],
           [ 0,  6, 10,  6],
           [ 0,  5,  7, 10]]
scores = []
for f, p in zip(fraction, ballots):
    scores.extend(int(n_voters*f)*[p])
    
# Convert into a numpy array
scores = np.array(scores)

winner = election.choose_score_voting(scores)
print('Score Voting: Winner {}'.format(choices[winner[0]]))

Score Voting: Winner Nashville


## Majority Judgment

This is another cardinal voting system.  In this case, the winner is determined based on who has the highest median ranking.  In the likely case of ties, the tie is broken by removing voters whose score is equal to the common median and recalculating the median until only one candidate has the best median.  [Wikipedia](https://en.wikipedia.org/wiki/Majority_judgment).

In [9]:
ballots = [[4, 2, 1, 1],
           [1, 4, 2, 2],
           [1, 2, 4, 3],
           [1, 2, 3, 4]]
scores = []
for f, p in zip(fraction, ballots):
    scores.extend(int(n_voters*f)*[p])
    
# Convert into a numpy array
scores = np.array(scores)

winner = election.choose_majority_judgment(scores)
print('Majority Judgment: Winner {}'.format(choices[winner[0]]))

Majority Judgment: Winner Nashville


## STAR Voting

This is a method of cardinal voting where after scoring all candidates, the top two in total score have an automatic run-off where the vote for each ballot is determined by which candidate had the higher score.  [Wikipedia](https://en.wikipedia.org/wiki/STAR_voting).

In [10]:
ballots = [[5, 2, 1, 0],
           [0, 5, 2, 1],
           [0, 3, 5, 3],
           [0, 2, 4, 5]]
scores = []
for f, p in zip(fraction, ballots):
    scores.extend(int(n_voters*f)*[p])
    
# Convert into a numpy array
scores = np.array(scores)

winner = election.choose_star(scores)
print('STAR: Winner {}'.format(choices[winner[0]]))

STAR: Winner Nashville


## STV Voting
Single Transferrable Vote (STV) voting is really, really complicated, but the basic idea is the same as IRV except that you also move excess votes above the minimum neede for a candidate to win on to the latter choices on the ballots.  This form of voting really only makes sense for multi-seat elections.  Otherwise, it should just be IRV.  [Wikipedia](https://en.wikipedia.org/wiki/Single_transferable_vote).

Start with a basic check to see that we get the same result as IRV on the Tennessee example.

In [11]:
winner = election.choose_stv(votes)
print('STV: Winner {}'.format(choices[winner[0]]))

STV: Winner Knoxville


Now I need to try a [non-trival example](https://en.wikipedia.org/wiki/Single_transferable_vote#Example) to see that I get the right answer.

In [12]:
foods = ['Oranges', 'Pears', 'Chocolate', 'Strawberries', 'Hamburgers']
food_votes = np.array(4*[[1,0,0,0,0]]+
                      2*[[2,1,0,0,0]]+
                      8*[[0,0,1,2,0]]+
                      4*[[0,0,1,0,2]]+
                      [[0,0,0,1,0],[0,0,0,0,1]])
food_winner = election.choose_stv(food_votes,3)
print('The party menu is {}'.format(', '.join([foods[x] for x in food_winner])))

The party menu is Chocolate, Oranges, Strawberries


# Validation
Now let's do some simple validation just to look for sitations that will break the voting.  Let's start by breaking our algorithms down into basic categories.

In [35]:
single_winner_ordinal = [election.choose_fptp,
                         election.choose_irv,
                         election.choose_condorcet,
                         election.choose_schulze,
                         election.choose_borda]

multi_winner_ordinal = [election.choose_fptp,
                        election.choose_schulze,
                        election.choose_borda,
                        election.choose_stv]

single_winner_cardinal = [election.choose_score_voting,
                          election.choose_majority_judgment,
                          election.choose_star]

multi_winner_cardinal = [election.choose_score_voting]

## One-Vote Test

Make sure the voting algorithm doesn't break if there's just one vote cast.

In [36]:
test_choices = ['A','B','C','D','E']

test_votes = np.array([[1,2,3,4,5]])

print('Single-Winner Ordinal Methods:')
for method in single_winner_ordinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_votes)])))
    
print('\nMulti-Winner Ordinal Methods:')
for method in multi_winner_ordinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_votes,3)])))

test_score = np.array([[10,8,6,4,2]])

print('\nSingle-Winner Cardinal Methods:')
for method in single_winner_cardinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_score)])))
    
print('\nMulti-Winner Cardinal Methods:')
for method in multi_winner_cardinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_score,3)])))

Single-Winner Ordinal Methods:
choose_fptp -> Winners: A
choose_irv -> Winners: A
choose_condorcet -> Winners: A
choose_schulze -> Winners: A
choose_borda -> Winners: A

Multi-Winner Ordinal Methods:
choose_fptp -> Winners: C, B, A
choose_schulze -> Winners: A, B, C
choose_borda -> Winners: A, B, C
choose_stv -> Winners: A, B, C

Single-Winner Cardinal Methods:
choose_score_voting -> Winners: A
choose_majority_judgment -> Winners: A
choose_star -> Winners: A

Multi-Winner Cardinal Methods:
choose_score_voting -> Winners: A, B, C


## Tie Test
Let's try a case where A, B, and C are tied.

In [45]:
test_choices = ['A','B','C','D','E']

test_votes = np.array(30*[[1,2,3,4,5]]+
                      30*[[3,1,2,4,5]]+
                      30*[[2,3,1,4,5]])

print('Single-Winner Ordinal Methods:')
for method in single_winner_ordinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_votes)])))
    
print('\nMulti-Winner Ordinal Methods:')
for method in multi_winner_ordinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_votes,2)])))

test_score = np.array(30*[[10,8,6,4,2]]+
                      30*[[6,10,8,4,2]]+
                      30*[[8,6,10,4,2]])

print('\nSingle-Winner Cardinal Methods:')
for method in single_winner_cardinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_score)])))
    
print('\nMulti-Winner Cardinal Methods:')
for method in multi_winner_cardinal:
    print('{} -> Winners: {}'.format(method.__name__,', '.join([test_choices[x] for x in method(test_score,2)])))

Single-Winner Ordinal Methods:
choose_fptp -> Winners: C, B, A
choose_irv -> Winners: A, B, C
choose_condorcet -> Winners: A, B, C
choose_schulze -> Winners: C, B, A
choose_borda -> Winners: C, B, A

Multi-Winner Ordinal Methods:
choose_fptp -> Winners: C, B, A
choose_schulze -> Winners: C, B, A
choose_borda -> Winners: C, B, A
choose_stv -> Winners: A, B, C

Single-Winner Cardinal Methods:
choose_score_voting -> Winners: C, B, A
choose_majority_judgment -> Winners: A, B, C
choose_star -> Winners: A, B, C

Multi-Winner Cardinal Methods:
choose_score_voting -> Winners: C, B, A


## Random Voters Test
Now let's make a bunch of randomly distributed votes that are, on average, a tie, and see whether the winners of the elections are uniform.

In [75]:
test_choices = ['A','B','C','D','E']
vote_opts = np.array(range(1,len(test_choices)+1))

# Counters for the results of different methods
from collections import Counter
counter_swo = []
for i in range(len(single_winner_ordinal)):
    counter_swo.append(Counter())

counter_mwo = []
for i in range(len(multi_winner_ordinal)):
    counter_mwo.append(Counter())

# Let's run some experiments
n_trials = 100000
n_votes = 100

for i in range(n_trials):
    
    # Generate the votes
    votes = np.empty((n_votes,len(test_choices)),dtype=int)
    for j in range(n_votes):
        votes[j,:]=np.random.permutation(vote_opts)
    
    # Now, do the voting
    for method,counter in zip(single_winner_ordinal,counter_swo):
        winners = method(votes)
        if len(winners)==1:
            counter[test_choices[winners[0]]]+=1
        else:
            counter['{}-Way Tie'.format(len(winners))]+=1

    for method,counter in zip(multi_winner_ordinal,counter_mwo):
        winners = method(votes,3)
        if len(winners)==3:
            counter[''.join(sorted([test_choices[x] for x in winners]))]+=1
        else:
            counter['{}-Way Tie'.format(len(winners))]+=1
            
# Let's make a table for the single-winner methods

# First, we need the headers
header_single = set()
header_ties = set()
for counter in counter_swo:
    for k in counter.keys():
        if '-Way Tie' in k:
            header_ties.add(k)
        else:
            header_single.add(k)

# Now turn the header sets into a header list
header = sorted(header_single) + sorted(header_ties)

# Then make the output
output = '#### Single-Winner Voting Methods\n'
output += '|Method|'+'|'.join(header)+'|\n'
output += '|---|'+'|'.join(len(header)*['---'])+'|\n'
for method,counter in zip(single_winner_ordinal,counter_swo):
    output += '|'+method.__name__+'|'+'|'.join(['{:.1f}%'.format(counter[x]*100/n_trials) for x in header])+'|\n'

# Do it again for the multi winners.  First, we need the headers
header_single = set()
header_ties = set()
for counter in counter_mwo:
    for k in counter.keys():
        if '-Way Tie' in k:
            header_ties.add(k)
        else:
            header_single.add(k)

# Now turn the header sets into a header list
header = sorted(header_single) + sorted(header_ties)

# Then make the output
output += '#### Multi-Winner Voting Methods\n'
output += '|Method|'+'|'.join(header)+'|\n'
output += '|---|'+'|'.join(len(header)*['---'])+'|\n'
for method,counter in zip(multi_winner_ordinal,counter_mwo):
    output += '|'+method.__name__+'|'+'|'.join(['{:.1f}%'.format(counter[x]*100/n_trials) for x in header])+'|\n'

from IPython.display import display, Markdown

display(Markdown(output))

#### Single-Winner Voting Methods
|Method|A|B|C|D|E|2-Way Tie|3-Way Tie|4-Way Tie|5-Way Tie|
|---|---|---|---|---|---|---|---|---|---|
|choose_fptp|17.4%|17.9%|17.7%|17.7%|17.5%|10.6%|1.1%|0.1%|0.0%|
|choose_irv|17.4%|17.9%|17.7%|17.7%|17.5%|10.6%|1.1%|0.1%|0.0%|
|choose_condorcet|15.0%|15.1%|15.1%|15.1%|14.7%|20.0%|4.3%|0.7%|0.0%|
|choose_schulze|18.6%|18.7%|18.7%|18.6%|18.3%|6.7%|0.4%|0.0%|0.0%|
|choose_borda|19.2%|19.3%|19.5%|19.3%|19.0%|3.5%|0.1%|0.0%|0.0%|
#### Multi-Winner Voting Methods
|Method|ABC|ABD|ABE|ACD|ACE|ADE|BCD|BCE|BDE|CDE|4-Way Tie|5-Way Tie|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|choose_fptp|8.6%|8.7%|8.5%|8.6%|8.7%|8.6%|8.6%|8.6%|8.5%|8.5%|13.1%|0.8%|
|choose_schulze|8.8%|8.6%|8.7%|8.8%|8.9%|8.7%|8.9%|8.7%|8.7%|8.7%|11.6%|0.9%|
|choose_borda|9.6%|9.4%|9.5%|9.6%|9.6%|9.5%|9.6%|9.4%|9.4%|9.4%|5.0%|0.1%|
|choose_stv|9.9%|10.0%|9.9%|9.9%|9.8%|9.8%|10.0%|9.9%|9.8%|10.0%|0.2%|0.9%|


In [78]:
test_choices = ['A','B','C','D','E']

# Counters for the results of different methods
counter_swc = []
for i in range(len(single_winner_cardinal)):
    counter_swc.append(Counter())

counter_mwc = []
for i in range(len(multi_winner_cardinal)):
    counter_mwc.append(Counter())

# Let's run some experiments
n_trials = 100000
n_votes = 100

for i in range(n_trials):
    
    # Generate the votes
    votes = np.random.default_rng().integers(1,11,(n_votes,len(test_choices)))
    
    # Now, do the voting
    for method,counter in zip(single_winner_cardinal,counter_swc):
        winners = method(votes)
        if len(winners)==1:
            counter[test_choices[winners[0]]]+=1
        else:
            counter['{}-Way Tie'.format(len(winners))]+=1

    for method,counter in zip(multi_winner_cardinal,counter_mwc):
        winners = method(votes,3)
        if len(winners)==3:
            counter[''.join(sorted([test_choices[x] for x in winners]))]+=1
        else:
            counter['{}-Way Tie'.format(len(winners))]+=1
            
# Let's make a table for the single-winner methods

# First, we need the headers
header_single = set()
header_ties = set()
for counter in counter_swc:
    for k in counter.keys():
        if '-Way Tie' in k:
            header_ties.add(k)
        else:
            header_single.add(k)

# Now turn the header sets into a header list
header = sorted(header_single) + sorted(header_ties)

# Then make the output
output = '#### Single-Winner Voting Methods\n'
output += '|Method|'+'|'.join(header)+'|\n'
output += '|---|'+'|'.join(len(header)*['---'])+'|\n'
for method,counter in zip(single_winner_cardinal,counter_swc):
    output += '|'+method.__name__+'|'+'|'.join(['{:.1f}%'.format(counter[x]*100/n_trials) for x in header])+'|\n'

# Do it again for the multi winners.  First, we need the headers
header_single = set()
header_ties = set()
for counter in counter_mwc:
    for k in counter.keys():
        if '-Way Tie' in k:
            header_ties.add(k)
        else:
            header_single.add(k)

# Now turn the header sets into a header list
header = sorted(header_single) + sorted(header_ties)

# Then make the output
output += '#### Multi-Winner Voting Methods\n'
output += '|Method|'+'|'.join(header)+'|\n'
output += '|---|'+'|'.join(len(header)*['---'])+'|\n'
for method,counter in zip(multi_winner_cardinal,counter_mwc):
    output += '|'+method.__name__+'|'+'|'.join(['{:.1f}%'.format(counter[x]*100/n_trials) for x in header])+'|\n'

from IPython.display import display, Markdown

display(Markdown(output))

#### Single-Winner Voting Methods
|Method|A|B|C|D|E|2-Way Tie|3-Way Tie|4-Way Tie|5-Way Tie|
|---|---|---|---|---|---|---|---|---|---|
|choose_score_voting|19.4%|19.9%|19.6%|19.7%|19.5%|1.9%|0.0%|0.0%|0.0%|
|choose_majority_judgment|6.5%|6.5%|6.6%|6.5%|6.4%|29.7%|23.8%|10.1%|3.9%|
|choose_star|18.7%|19.1%|18.8%|18.9%|18.6%|5.9%|0.0%|0.0%|0.0%|
#### Multi-Winner Voting Methods
|Method|ABC|ABD|ABE|ACD|ACE|ADE|BCD|BCE|BDE|CDE|4-Way Tie|5-Way Tie|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
|choose_score_voting|9.7%|9.8%|9.9%|9.8%|9.6%|9.6%|9.7%|9.6%|9.7%|9.7%|2.8%|0.0%|
